### LIBRARIES

In [1]:
import time
import numpy as np
import pandas as pd
import datetime as dt

from matplotlib import pyplot as plt
import seaborn as sns

import scipy.stats as st
import empiricaldist as emd
import thinkstats as ts
import utils as ut

from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedStratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score, precision_recall_curve, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.calibration import CalibrationDisplay, CalibratedClassifierCV

import warnings

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
warnings.filterwarnings('ignore')

### CLEANING AND VALIDATION

In [3]:
original_data = pd.read_csv('ecom_data.csv', encoding='unicode_escape', parse_dates=['InvoiceDate'])

df = original_data.copy()
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [4]:
print(df.InvoiceDate.min(), df.InvoiceDate.max())

2010-12-01 08:26:00 2011-12-09 12:50:00


In [5]:
df.info() # there are some nulls in Description and CustomerID

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [7]:
pd.DataFrame(df.describe([.003, .01, .025, .05, .975, .99, .997])).T

,count,mean,min,0.3%,1%,2.5%,5%,50%,97.5%,99%,99.7%,max,std
Quantity,541909.000,9.552,-80995.000,-22.000,-2.000,1.000,1.000,3.000,48.000,100.000,240.000,80995.000,218.081
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2010-12-01 14:32:00,2010-12-03 11:13:00,2010-12-06 16:57:00,2010-12-13 09:35:00,2011-07-19 17:17:00,2011-12-05 17:24:00,2011-12-08 09:28:00,2011-12-09 08:39:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.000,4.611,-11062.060,0.000,0.190,0.390,0.420,2.080,12.750,18.000,39.950,38970.000,96.760
CustomerID,406829.000,15287.691,12346.000,12362.000,12415.000,12476.000,12626.000,15152.000,18109.000,18212.000,18262.000,18287.000,1713.600


In [8]:
# Investigating observations with unit price less than 0

df.loc[df['UnitPrice'] < 0, :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.060,NaN,United Kingdom
299984,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.060,NaN,United Kingdom


In [9]:
# That A in front of the InvoiceNo stands for "adjust", probably. I'm gonna drop these two

print(len(df))

df = df.loc[df['UnitPrice'] >= 0, :]

print(len(df))

541909
541907


In [10]:
# What does it mean for the price to be zero?

df.loc[df['UnitPrice'] == 0, :].head(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.000,NaN,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.000,NaN,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.000,NaN,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.000,NaN,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.000,NaN,United Kingdom
1988,536550,85044,NaN,1,2010-12-01 14:34:00,0.000,NaN,United Kingdom
2024,536552,20950,NaN,1,2010-12-01 14:34:00,0.000,NaN,United Kingdom
2025,536553,37461,NaN,3,2010-12-01 14:35:00,0.000,NaN,United Kingdom
2026,536554,84670,NaN,23,2010-12-01 14:35:00,0.000,NaN,United Kingdom
2406,536589,21777,NaN,-10,2010-12-01 16:50:00,0.000,NaN,United Kingdom


In [11]:
df.loc[df['UnitPrice'] == 0, 'CustomerID'].value_counts(dropna=False)

CustomerID
NaN          2475
13081.000       4
14646.000       4
14911.000       2
13985.000       2
12415.000       2
16560.000       1
15107.000       1
13239.000       1
13113.000       1
12457.000       1
14410.000       1
17667.000       1
16818.000       1
17560.000       1
12647.000       1
15581.000       1
12507.000       1
12748.000       1
16133.000       1
12446.000       1
18059.000       1
14110.000       1
15804.000       1
12437.000       1
12431.000       1
13014.000       1
15602.000       1
12603.000       1
12444.000       1
16406.000       1
13256.000       1
Name: count, dtype: int64

In [12]:
# It's tough to make a meaning out of those, many of them does not even have a CustomerID.

df.loc[df['UnitPrice'] == 0, :].shape

(2515, 8)

In [13]:
# I'm dropping those as well, since I don't have necessary information to make sense at first glance.

df = df.loc[df['UnitPrice'] > 0, :]

df.shape

(539392, 8)

In [14]:
# Let's check out invoices that have 'C' on their identifier, it stands for "cancel" I suppose.

df.loc[df['InvoiceNo'].str.contains('C'), :].head(30)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.500,14527.000,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.650,15311.000,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.650,17548.000,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.290,17548.000,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.290,17548.000,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.290,17548.000,United Kingdom
239,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.450,17548.000,United Kingdom
240,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.650,17548.000,United Kingdom
241,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.650,17548.000,United Kingdom
939,C536506,22960,JAM MAKING SET WITH JARS,-6,2010-12-01 12:38:00,4.250,17897.000,United Kingdom


In [15]:
df.loc[(df['InvoiceNo'].str.startswith('C')) & (df['Quantity'] >= 0), :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [16]:
print(df.loc[df['InvoiceNo'].str.contains('C'), 'InvoiceDate'].min(), df.loc[df['InvoiceNo'].str.contains('C'), 'InvoiceDate'].max())

2010-12-01 09:41:00 2011-12-09 11:58:00


While creating variables, I need to make sure that I have that data available chronologically. To not deal with it immediately, and to set the baseline, I may start with creating variables regarding invoices.

In [17]:
# I'll make quantities and unitprices > 0 by taking their absolute, since I can already identify canceled orders via their InvoiceNo

df['Quantity'] = abs(df['Quantity'])

df.loc[df['UnitPrice'] < 0, :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


### DATA PROCESSING

In [18]:
new_df = pd.DataFrame(df.InvoiceNo.unique().tolist()).rename(columns={0:'InvoiceNo'})
new_df.head()

,InvoiceNo
0,536365
1,536366
2,536367
3,536368
4,536369


In [19]:
new_df['returned'] = 0

returned_invs = df.loc[df['InvoiceNo'].str.contains('C'), 'InvoiceNo'].unique().tolist()
new_df.loc[new_df.InvoiceNo.isin(returned_invs), 'returned'] = 1

new_df.returned.value_counts()

returned
0    19960
1     3836
Name: count, dtype: int64

Before creating variables, as I understand:

* InvoiceNo: Unique identifier of the invoice
* StockCode: Unique identifier of the item
* Description: Description of the item
* Quantity: Amount of that item that has been included in the order
* InvoiceDate: Date of the invoice
* UnitPrice: Price value of one unit of that item
* CustomerID: Unique identifier of the customer
* Country: Where the order has been created from

In [20]:
len(new_df) == df.loc[:, ['InvoiceNo', 'Country']].drop_duplicates().shape[0]

True

In [21]:
new_df = new_df.merge(df.loc[:, ['InvoiceNo', 'Country']].drop_duplicates(), how='left', on=['InvoiceNo'])

new_df.head()

,InvoiceNo,returned,Country
0,536365,0,United Kingdom
1,536366,0,United Kingdom
2,536367,0,United Kingdom
3,536368,0,United Kingdom
4,536369,0,United Kingdom


In [22]:
new_df.Country.unique()

# There seems to be a lot of countries, I might go for something like target encoding or count encoding to not go off the charts with dimensionality.

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [23]:
df['total_price'] = df['Quantity'] * df['UnitPrice'] # price of an item x how many did the customer order

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340


In [24]:
df.loc[df['StockCode'].str.contains('A'), :].head(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.550,17850.000,United Kingdom,15.300
66,536375,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.550,17850.000,United Kingdom,15.300
100,536378,84519A,TOMATO CHARLIE+LOLA COASTER SET,6,2010-12-01 09:37:00,2.950,14688.000,United Kingdom,17.700
120,536381,37444A,YELLOW BREAKFAST CUP AND SAUCER,1,2010-12-01 09:41:00,2.950,15311.000,United Kingdom,2.950
172,536385,85049A,TRADITIONAL CHRISTMAS RIBBONS,12,2010-12-01 09:56:00,1.250,17420.000,United Kingdom,15.000
203,536389,85014A,BLACK/BLUE POLKADOT UMBRELLA,3,2010-12-01 10:03:00,5.950,12431.000,Australia,17.850
220,536390,85123A,WHITE HANGING HEART T-LIGHT HOLDER,64,2010-12-01 10:19:00,2.550,17511.000,United Kingdom,163.200
262,536394,85123A,WHITE HANGING HEART T-LIGHT HOLDER,32,2010-12-01 10:39:00,2.550,13408.000,United Kingdom,81.600
278,536396,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 10:51:00,2.550,17850.000,United Kingdom,15.300


In [25]:
# Some of the stockcodes include characters in addition to numbers, 
# I want to check if they specify a category.

df.loc[df['StockCode'].str.contains('B'), :].head(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000
51,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.750,17850.000,United Kingdom,22.000
68,536375,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:32:00,2.750,17850.000,United Kingdom,22.000
90,536378,84997B,RED 3 PIECE RETROSPOT CUTLERY SET,12,2010-12-01 09:37:00,3.750,14688.000,United Kingdom,45.000
101,536378,85183B,CHARLIE & LOLA WASTEPAPER BIN FLORA,48,2010-12-01 09:37:00,1.250,14688.000,United Kingdom,60.000
102,536378,85071B,RED CHARLIE+LOLA PERSONAL DOORSIGN,96,2010-12-01 09:37:00,0.380,14688.000,United Kingdom,36.480
132,536381,15056BL,EDWARDIAN PARASOL BLACK,2,2010-12-01 09:41:00,5.950,15311.000,United Kingdom,11.900
177,536386,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01 09:57:00,1.650,16029.000,United Kingdom,165.000
202,536389,85014B,RED RETROSPOT UMBRELLA,6,2010-12-01 10:03:00,5.950,12431.000,Australia,35.700
234,536390,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01 10:19:00,1.650,17511.000,United Kingdom,165.000


In [26]:
# Well as I can see, I'll have to specify a pattern because just saying "include B" also includes stockcodes that have
# characters after the specified character. Endswith wouldn't work as well since it'd also include the characters before it.

df.loc[df['StockCode'].str.contains(r'[A-Z]'), :].head(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.000,12583.000,France,54.000
49,536373,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.550,17850.000,United Kingdom,15.300
51,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.750,17850.000,United Kingdom,22.000
60,536373,82494L,WOODEN FRAME ANTIQUE WHITE,6,2010-12-01 09:02:00,2.550,17850.000,United Kingdom,15.300
61,536373,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 09:02:00,3.390,17850.000,United Kingdom,20.340
62,536373,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 09:02:00,3.390,17850.000,United Kingdom,20.340


In [27]:
df['stock_code_char'] = df['StockCode'].str.extract(r'([A-Z]+)')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price,stock_code_char
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300,A
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000,B
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,G
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,E


In [28]:
df.loc[df['stock_code_char'] == 'POST', :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price,stock_code_char
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.000,12583.000,France,54.000,POST
386,536403,POST,POSTAGE,1,2010-12-01 11:27:00,15.000,12791.000,Netherlands,15.000,POST
1123,536527,POST,POSTAGE,1,2010-12-01 13:04:00,18.000,12662.000,Germany,18.000,POST
5073,536840,POST,POSTAGE,1,2010-12-02 18:27:00,18.000,12738.000,Germany,18.000,POST
5258,536852,POST,POSTAGE,1,2010-12-03 09:51:00,18.000,12686.000,France,18.000,POST


Within the same order, there can be more than category of items. I may come up with a lot of variables out of this, and see if they have any effect on the order cancellation.

In [29]:
df['stock_code_char'] = df['stock_code_char'].fillna('NONE')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price,stock_code_char
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300,A
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,NONE
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000,B
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,G
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,E


In [30]:
stock_code_counts = df.groupby('InvoiceNo').agg({
    'stock_code_char':'value_counts'
}).rename(columns={'stock_code_char':'counts'}).reset_index()

stock_code_counts.head()

,InvoiceNo,stock_code_char,counts
0,536365,NONE,3
1,536365,A,1
2,536365,B,1
3,536365,E,1
4,536365,G,1


In [31]:
stock_code_pvt = pd.pivot_table(stock_code_counts, index='InvoiceNo', values='counts', columns='stock_code_char').reset_index()
stock_code_pvt.head()

stock_code_char,InvoiceNo,A,AMAZONFEE,B,BANK,BL,C,CRUK,D,DCGS,DCGSSBOY,DCGSSGIRL,DOT,E,F,G,H,I,J,K,L,M,N,NONE,O,P,PADS,POST,R,S,T,U,V,W,Y,Z
0,536365,1.000,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,536366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,536367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,536368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,536369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
stock_code_pvt.fillna(0, inplace=True)

In [33]:
tmp = df.groupby('InvoiceNo').agg({
    'StockCode':'nunique', # number of unique items in the order
    'Quantity':'sum', # total quantity in the order
    'total_price':'sum' # total price of the order
}).reset_index()

tmp.head()

,InvoiceNo,StockCode,Quantity,total_price
0,536365,7,40,139.120
1,536366,2,12,22.200
2,536367,12,83,278.730
3,536368,4,15,70.050
4,536369,1,3,17.850


In [34]:
new_df = new_df.merge(tmp, how='left', on=['InvoiceNo'])

del tmp

new_df.head()

,InvoiceNo,returned,Country,StockCode,Quantity,total_price
0,536365,0,United Kingdom,7,40,139.120
1,536366,0,United Kingdom,2,12,22.200
2,536367,0,United Kingdom,12,83,278.730
3,536368,0,United Kingdom,4,15,70.050
4,536369,0,United Kingdom,1,3,17.850


In [35]:
new_df.rename(columns = {
    'StockCode':'nunique_items',
    'Quantity':'total_quantity'
}, inplace=True)

new_df.head()

,InvoiceNo,returned,Country,nunique_items,total_quantity,total_price
0,536365,0,United Kingdom,7,40,139.120
1,536366,0,United Kingdom,2,12,22.200
2,536367,0,United Kingdom,12,83,278.730
3,536368,0,United Kingdom,4,15,70.050
4,536369,0,United Kingdom,1,3,17.850


In [36]:
new_df = new_df.merge(stock_code_pvt, how='left', on=['InvoiceNo'])

new_df.head()

,InvoiceNo,returned,Country,nunique_items,total_quantity,total_price,A,AMAZONFEE,B,BANK,BL,C,CRUK,D,DCGS,DCGSSBOY,DCGSSGIRL,DOT,E,F,G,H,I,J,K,L,M,N,NONE,O,P,PADS,POST,R,S,T,U,V,W,Y,Z
0,536365,0,United Kingdom,7,40,139.120,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,536366,0,United Kingdom,2,12,22.200,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,536367,0,United Kingdom,12,83,278.730,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,12.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,536368,0,United Kingdom,4,15,70.050,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,536369,0,United Kingdom,1,3,17.850,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [37]:
# A quick validation

new_df['nunique_items'].head(15) == new_df[[col for col in new_df.columns if col not in ['InvoiceNo', 'returned', 'Country', 'nunique_items', 'total_quantity', 'total_price']]].head(15).sum(axis=1)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
dtype: bool

In [38]:
def create_dt_feats(dataframe):
    dataframe['year'] = dataframe.InvoiceDate.dt.year
    dataframe['month'] = dataframe.InvoiceDate.dt.month
    dataframe['day_of_month'] = dataframe.InvoiceDate.dt.day
    dataframe['month_start'] = dataframe.InvoiceDate.dt.is_month_start.astype(int)
    dataframe['month_end'] = dataframe.InvoiceDate.dt.is_month_end.astype(int)
    dataframe['weekend'] = dataframe.InvoiceDate.dt.weekday // 4
    dataframe['day_of_year'] = dataframe.InvoiceDate.dt.dayofyear
    
    return dataframe

In [39]:
df = create_dt_feats(df)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price,stock_code_char,year,month,day_of_month,month_start,month_end,weekend,day_of_year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,15.300,A,2010,12,1,1,0,0,335
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,NONE,2010,12,1,1,0,0,335
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,22.000,B,2010,12,1,1,0,0,335
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,G,2010,12,1,1,0,0,335
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,20.340,E,2010,12,1,1,0,0,335


In [40]:
new_df = new_df.merge(df.loc[:, 
                             ['InvoiceNo', 'InvoiceDate', 'year', 'month', 'day_of_month', 'month_start', 'month_end', 'weekend', 'day_of_year']].drop_duplicates(),
                             how = 'left', on=['InvoiceNo'])

new_df.head()

,InvoiceNo,returned,Country,nunique_items,total_quantity,total_price,A,AMAZONFEE,B,BANK,BL,C,CRUK,D,DCGS,DCGSSBOY,DCGSSGIRL,DOT,E,F,G,H,I,J,K,L,M,N,NONE,O,P,PADS,POST,R,S,T,U,V,W,Y,Z,InvoiceDate,year,month,day_of_month,month_start,month_end,weekend,day_of_year
0,536365,0,United Kingdom,7,40,139.120,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2010-12-01 08:26:00,2010,12,1,1,0,0,335
1,536366,0,United Kingdom,2,12,22.200,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2010-12-01 08:28:00,2010,12,1,1,0,0,335
2,536367,0,United Kingdom,12,83,278.730,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,12.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2010-12-01 08:34:00,2010,12,1,1,0,0,335
3,536368,0,United Kingdom,4,15,70.050,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2010-12-01 08:34:00,2010,12,1,1,0,0,335
4,536369,0,United Kingdom,1,3,17.850,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2010-12-01 08:35:00,2010,12,1,1,0,0,335
